<a href="https://colab.research.google.com/github/viniciusrgf/Projeto_tcc/blob/main/Modelo_def_tcc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Treinamento Modelo #1 Tcc: Criação de modelo capaz de analisar e desenvolver testes simples Web

Etapa #1: Requerimentos do Modelo - Modelo deve analisar uma pagina web e identificar elementos comuns, como caixas de texto e botões e desenvolver testes para esses elementos

Etapa #2: Coletar e rotular dados atraves de uma coleta dados de treinamento que possam ser usados ​​para treinar o modelo. Esses dados podem incluir screenshots de páginas da web, informações sobre os elementos da página e testes de exemplo que já foram escritos.

Etapa #3: Treinar esse modelo para poder conseguir prever se um novo 'teste' sem uma label é valido ou não

Site Exemplo: http://saucedemo.com/ https://testpages.herokuapp.com/styled/index.html https://theuselessweb.com/

IMPORTS

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from bs4 import BeautifulSoup
import requests
import pandas as pd
from google.colab import drive
import random
import csv
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from tabulate import tabulate

Creates the datasets

In [ ]:
def process_log_file(log_file, subset_ratio):
    modified_file = "modified_file.log" # Arquivo gerado contendo apenas as linhas que foram alteradas.
    subset_file = "subset_file.log" # Arquivo que contem as linhas não alteradas e as linhas alteradas.
    output_file_1 = "dataset_1.csv" # Arquivo de saída para dataset_1 dividido em 6 categorias e com labels.
    output_file_2 = "dataset_2.csv" # Arquivo de saída para dataset_2 dividido em 6 categorias.

    urls = [
        "https://www.saucedemo.com/cart.html",
        "https://www.saucedemo.com/inventory-item.html",
        "https://www.saucedemo.com/inventory.html",
        "https://www.saucedemo.com/login.html",
        "https://www.saucedemo.com/",
        "https://www.saucedemo.com/inventory-item.html"
    ]

    modified_lines = []
    subset_indexes = []

    # Abre o arquivo de log original
    with open(log_file, "r") as f_log:
        lines = f_log.readlines()
        subset_size = int(len(lines) * subset_ratio)
        subset_indexes = random.sample(range(len(lines)), subset_size)

        # Itera pelas linhas do arquivo de log
        for i, line in enumerate(lines):
            # Verifica se o índice da linha está no conjunto de índices do subconjunto
            if i in subset_indexes:
                parts = line.split()
                # Se o numero aleatorio cair em 10, o usuairo muda apenas o numero de elementos na linha
                mod_type = random.randint(1, 10)
                if mod_type == 10:
                    num_elements = random.randint(1, 150)
                    parts[4] = str(num_elements)
                else:
                    # Se não, ele muda dois parametros da linha, para erros aleatorios.
                    num_elements = random.randint(1, 150)
                    parts[4] = str(num_elements)
                    page = random.choice(urls)
                    parts[-1] = page
                new_line = " ".join(parts) + "\n"
                modified_lines.append(new_line)
            else:
                modified_lines.append(line)

    # Escreve as linhas modificadas no arquivo de subconjunto
    with open(subset_file, "w") as f_subset:
        f_subset.writelines(modified_lines[i] for i in subset_indexes)

    # Escreve todas as linhas modificadas no arquivo modificado
    with open(modified_file, "w") as f:
        f.writelines(modified_lines)

    # Abre o arquivo de subconjunto em modo de leitura
    with open(subset_file, "r") as f:
        subset_lines = f.readlines()
        subset_lines = [line.strip().split() for line in subset_lines]

    # Abre o arquivo de saída para dataset_1 em modo de escrita
    with open(output_file_1, 'w', newline='') as f:
        writer = csv.writer(f)

        # Escreve a linha de cabeçalho com as categorias, incluindo a categoria adicional 'label'
        writer.writerow(['time', 'date', 'element', 'action', 'n_elem', 'url', 'label'])

        # Abre o arquivo original em modo de leitura
        with open(modified_file, 'r') as f:
            modified_lines = f.readlines()
            modified_lines = [line.strip().split() for line in modified_lines]

            # Itera pelas linhas do arquivo original
            for line in modified_lines:
                # Verifica se a linha está presente no subconjunto
                if line[:5] in [l[:5] for l in subset_lines]:
                    # Escreve a linha com a categoria 'fault' adicionada
                    writer.writerow(line + ['fault'])
                else:
                    # Escreve a linha com a categoria 'valid' adicionada
                    writer.writerow(line + ['valid'])

    # Abre o arquivo de saída para dataset_2 em modo de escrita
    with open(output_file_2, 'w', newline='') as f:
        writer = csv.writer(f)

        # Escreve a linha de cabeçalho com as categorias
        writer.writerow(['time', 'date', 'element', 'action', 'n_elem', 'url'])

        # Abre o arquivo original em modo de leitura
        with open(modified_file, 'r') as f:
            modified_lines = f.readlines()
            modified_lines = [line.strip().split() for line in modified_lines]

            # Itera pelas linhas do arquivo original
            for line in modified_lines:
                # Escreve as categorias no arquivo de saída como uma nova linha
                writer.writerow(line)

log_file_1 = "dic.log"
subset_ratio_1 = 0.4

log_file_2 = "dic_2.log"
subset_ratio_2 = 0.2

process_log_file(log_file_1, subset_ratio_1)
process_log_file(log_file_2, subset_ratio_2)


trains the dataset with diferent parameters

In [ ]:
# Função para treinar um modelo e retornar as métricas de avaliação
def train_model(X_train, X_test, y_train, y_test, model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    return accuracy, precision, recall, roc_auc, conf_matrix

# Carregar os dados do arquivo 'dataset_1.csv' usando o pandas
data = pd.read_csv('dataset_1.csv')

# Remover as colunas 'date' e 'time' dos dados, pois não são relevantes
data = data.drop(['date', 'time'], axis=1)

# Converter colunas categóricas em valores numéricos nos dados
for col in data.columns:
    if data[col].dtype == object:
        data[col] = pd.Categorical(data[col]).codes.astype(float)

# Aplicar codificação one-hot na coluna 'action' nos dados
data = pd.get_dummies(data, columns=['action'])

# Converter a coluna 'url' em valores numéricos nos dados
data['url'] = pd.Categorical(data['url']).codes.astype(float)

# Substituir os rótulos 'valid' por 0 e 'fault' por 1 na coluna 'label' dos dados
data['label'] = data['label'].replace({'valid': 0, 'fault': 1})

# Separar as features (X) e os rótulos (y) dos dados
X = data.drop(['label'], axis=1)
y = data['label']

# Dividir os dados em conjuntos de treinamento e teste usando train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar o modelo de Regressão Logística isoladamente
lr_model = LogisticRegression(random_state=42)
lr_accuracy, lr_precision, lr_recall, lr_roc_auc, lr_conf_matrix = train_model(X_train, X_test, y_train, y_test, lr_model)

# Treinar o modelo de Regressão Logística com Árvore de Decisão
lr_dt_model = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), algorithm='SAMME', random_state=42)
lr_dt_accuracy, lr_dt_precision, lr_dt_recall, lr_dt_roc_auc, lr_dt_conf_matrix = train_model(X_train, X_test, y_train, y_test, lr_dt_model)

# Treinar o modelo AdaBoost isoladamente
ada_model = AdaBoostClassifier(random_state=42)
ada_accuracy, ada_precision, ada_recall, ada_roc_auc, ada_conf_matrix = train_model(X_train, X_test, y_train, y_test, ada_model)

# Treinar o modelo AdaBoost com Árvore de Decisão
ada_dt_model = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), algorithm='SAMME', random_state=42)
ada_dt_accuracy, ada_dt_precision, ada_dt_recall, ada_dt_roc_auc, ada_dt_conf_matrix = train_model(X_train, X_test, y_train, y_test, ada_dt_model)

# Criar uma tabela para comparar os resultados
results = pd.DataFrame({
    'Modelo': ['Regressão Logística', 'Regressão Logística + Árvore de Decisão', 'AdaBoost', 'AdaBoost + Árvore de Decisão'],
    'Acurácia': [lr_accuracy, lr_dt_accuracy, ada_accuracy, ada_dt_accuracy],
    'Precisão': [lr_precision, lr_dt_precision, ada_precision, ada_dt_precision],
    'Recall': [lr_recall, lr_dt_recall, ada_recall, ada_dt_recall],
    'ROC AUC': [lr_roc_auc, lr_dt_roc_auc, ada_roc_auc, ada_dt_roc_auc],
    'Matriz de Confusão': [lr_conf_matrix, lr_dt_conf_matrix, ada_conf_matrix, ada_dt_conf_matrix]
})

print(results)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


                                 Model  Accuracy  Precision    Recall  \
0                  Logistic Regression  0.846154   0.875000  0.954545   
1  Logistic Regression + Decision Tree  0.961538   0.956522  1.000000   
2                             AdaBoost  0.961538   0.956522  1.000000   
3             AdaBoost + Decision Tree  0.961538   0.956522  1.000000   

    ROC AUC   Confusion Matrix  
0  0.602273  [[1, 3], [1, 21]]  
1  0.875000  [[3, 1], [0, 22]]  
2  0.875000  [[3, 1], [0, 22]]  
3  0.875000  [[3, 1], [0, 22]]  


try to guess from the dataset_2 that dont have labels

In [ ]:
def train_and_predict_logistic():
    # Carregar os dados de treinamento do arquivo 'dataset_1.csv'
    data_train = pd.read_csv('dataset_1.csv')

    # Remover as colunas 'date' e 'time' dos dados de treinamento
    data_train = data_train.drop(['date', 'time'], axis=1)

    # Converter colunas categóricas em valores numéricos nos dados de treinamento
    for col in data_train.columns:
        if data_train[col].dtype == object:
            data_train[col] = pd.Categorical(data_train[col]).codes.astype(float)

    # Aplicar codificação one-hot na coluna 'action' dos dados de treinamento
    data_train = pd.get_dummies(data_train, columns=['action'])

    # Converter a coluna 'url' em valores numéricos nos dados de treinamento
    data_train['url'] = pd.Categorical(data_train['url']).codes.astype(float)

    # Substituir 'valid' por 0 e 'fault' por 1 na coluna 'label' dos dados de treinamento
    data_train['label'] = data_train['label'].replace({'valid': 0, 'fault': 1})

    # Separar as features (X_train) e o rótulo (y_train) dos dados de treinamento
    X_train = data_train.drop(['label'], axis=1)
    y_train = data_train['label']

    # Criar um modelo de regressão logística e ajustá-lo aos dados de treinamento
    model = LogisticRegression(random_state=42)
    model.fit(X_train, y_train)

    # Carregar os dados de teste do arquivo 'dataset_2.csv'
    data_test = pd.read_csv('dataset_2.csv')

    # Remover as colunas 'date' e 'time' dos dados de teste
    data_test = data_test.drop(['date', 'time'], axis=1)

    # Converter colunas categóricas em valores numéricos nos dados de teste
    for col in data_test.columns:
        if data_test[col].dtype == object:
            data_test[col] = pd.Categorical(data_test[col]).codes.astype(float)

    # Aplicar codificação one-hot na coluna 'action' dos dados de teste
    data_test = pd.get_dummies(data_test, columns=['action'])

    # Converter a coluna 'url' em valores numéricos nos dados de teste
    data_test['url'] = pd.Categorical(data_test['url']).codes.astype(float)

    # Fazer previsões para os dados de teste usando o modelo treinado
    labels_predicted = model.predict(data_test)

    # Calcular a acurácia da previsão considerando rótulos 'valid' (0)
    accuracy = (labels_predicted == 0).mean() * 100

    return accuracy


def train_and_predict_adaboost():
    # Carregar os dados de treinamento do arquivo 'dataset_1.csv'
    data_train = pd.read_csv('dataset_1.csv')

    # Remover as colunas 'date' e 'time' dos dados de treinamento
    data_train = data_train.drop(['date', 'time'], axis=1)

    # Substituir 'valid' por 0 e 'fault' por 1 na coluna 'label' dos dados de treinamento
    data_train['label'] = data_train['label'].replace({'valid': 0, 'fault': 1})

    # Aplicar codificação one-hot nas colunas 'element', 'action' e 'url' dos dados de treinamento
    data_train = pd.get_dummies(data_train, columns=['element', 'action', 'url'])

    # Separar as features (X_train) e o rótulo (y_train) dos dados de treinamento
    X_train = data_train.drop(['label'], axis=1)
    y_train = data_train['label']

    # Definir o estimador base como DecisionTreeClassifier e criar um modelo AdaBoostClassifier
    base_estimator = DecisionTreeClassifier()
    model = AdaBoostClassifier(base_estimator=base_estimator, algorithm='SAMME')

    # Treinar o modelo usando os dados de treinamento
    model.fit(X_train, y_train)

    # Carregar os dados de teste do arquivo 'dataset_2.csv'
    data_test = pd.read_csv('dataset_2.csv')

    # Remover as colunas 'date' e 'time' dos dados de teste
    data_test = data_test.drop(['date', 'time'], axis=1)

    # Aplicar codificação one-hot nas colunas 'element', 'action' e 'url' dos dados de teste
    data_test = pd.get_dummies(data_test, columns=['element', 'action', 'url'])

    # Reindexar as colunas dos dados de teste para corresponder às colunas dos dados de treinamento, preenchendo com zeros
    data_test = data_test.reindex(columns=X_train.columns, fill_value=0)

    # Fazer previsões para os dados de teste
    labels_predicted = model.predict(data_test)

    # Calcular a acurácia do modelo considerando rótulos 'valid' (0)
    accuracy = (labels_predicted == 0).mean() * 100

    return accuracy


# Treinar e prever usando Regressão Logística
accuracy_logistic = train_and_predict_logistic()

# Treinar e prever usando AdaBoost com Decision Tree
accuracy_adaboost = train_and_predict_adaboost()

# Criar uma tabela para comparar as acurácias
table = [["Modelo", "Acurácia"],
         ["Regressão Logística", f"{accuracy_logistic:.2f}%"],
         ["AdaBoost com Decision Tree", f"{accuracy_adaboost:.2f}%"]]

# Exibir a tabela
print(tabulate(table, headers="firstrow", tablefmt="fancy_grid"))


╒═════════════════════════════╤════════════╕
│ Model                       │ Accuracy   │
╞═════════════════════════════╪════════════╡
│ Logistic Regression         │ 3.97%      │
├─────────────────────────────┼────────────┤
│ AdaBoost with Decision Tree │ 80.16%     │
╘═════════════════════════════╧════════════╛


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
